In [59]:
import pandas as pd
import sys
import os
import json
from bokeh.io import output_notebook, output_file, save
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.palettes import Category20
from bokeh.layouts import gridplot
from bokeh.models import HoverTool
import collections

In [3]:
# setup auto reload for working on the util module
%load_ext autoreload
%autoreload 2

In [2]:
# add the parent path so that utils can be imported
sys.path.insert(0, os.path.abspath('..'))

In [5]:
output_notebook()

Loading BokehJS ...

In [54]:
from vis.utils import read_rdf

In [55]:
dataframes = read_rdf('../metadata.json', char_types=['Aluminate Species'])

In [62]:
available_cmpds_dict = collections.defaultdict(list)

for frame in dataframes:
    # Set the new, aluminate species key
    available_cmpds_dict[frame.characteristics['Aluminate Species'][0]].extend(
        frame.characteristics['Inter-atom distances'])

In [63]:
available_cmpds_dict

defaultdict(list,
            {'(OH)2Al-O2-Al(OH)2- + 181 H2O + 2 Na+': ['Al-Ob', 'Al-OH'],
             '(OH)3Al-(OH)-Al(OH)3- + 179 H2O + HO- + 2 Na+': ['Al-Ob',
              'Al-OH'],
             '(OH)3Al-(OH)2 -Al(OH)3- + 179 H2O + 2 Na+': ['Al-Ob', 'Al-OH'],
             '(OH)3Al-O-Al(OH)32-+ 180 H2O + 2 Na+': ['Al-Ob', 'Al-OH']})

In [56]:
dataframes[0].head()
dataframes[0].characteristics

defaultdict(list,
            {'Aluminate Species': ['(OH)3Al-O-Al(OH)32-+ 180 H2O + 2 Na+'],
             'Inter-atom distances': ['Al-Ob', 'Al-OH'],
             'Simulated Data': ['Simulated RDF']})

In [57]:
# Create the base figure to plot to:
fig = figure()

# Create the hover tool
fig.add_tools(HoverTool(tooltips=[
    ("Radius", "@r"),
    ("Probability", "@{RDF_Al-Oh}"),
    ("Connectivity", "@{RCN_Al-Oh}")
]))

# Generate the colors to use, the number of dataframes is equal to the
# number of colored lines we want.
colors = Category20[len(dataframes)]

# Iterate through the list of dataframes
for (df, df_color) in zip(dataframes, colors):
    
    # Declare the source, this allows plots to be linked.
    fig_source = ColumnDataSource(df)
    
    # Declare the actual line plot.
    fig.line(
        source = fig_source,
        x='r',
        y='RDF_Al-Oh',
        legend=df.characteristics['Aluminate Species'][0],
        color=df_color,
        line_width=1.5
    )

# Display the figure.
show(fig)

## Linked RDF Plots

In [38]:
# Create the base figure to plot to:
left_fig = figure(title="RDF of Al-OH", plot_width=400)
right_fig = figure(title="RDF of Al-O (bending)", plot_width=400)

# Add the hover tool
left_fig.add_tools(HoverTool(tooltips=[
    ("Radius", "@r"),
    ("Probability", "@{RDF_Al-Oh}"),
    ("Connectivity", "@{RCN_Al-Oh}")
]))

right_fig.add_tools(HoverTool(tooltips=[
    ("Radius", "@r"),
    ("Probability", "@{RDF_Al-Ob}"),
    ("Connectivity", "@{RCN_Al-Ob}")
]))

# Generate the colors to use, the number of dataframes is equal to the
# number of colored lines we want.
colors = Category20[len(dataframes)]

# Iterate through the list of dataframes
for (df, df_color) in zip(dataframes, colors):
    
    # Declare the source, this allows plots to be linked.
    fig_source = ColumnDataSource(df)
    
    # Declare the actual line plot.
    left_fig.line(
        source = fig_source,
        x='r',
        y='RDF_Al-Oh',
        legend='dimer',
        color=df_color,
        line_width=1.5
    )
    
    # Declare the actual line plot.
    right_fig.line(
        source = fig_source,
        x='r',
        y='RDF_Al-Ob',
        legend='dimer',
        color=df_color,
        line_width=1.5
    )
    
plot = gridplot([[left_fig, right_fig]])

# Display the figure.
show(plot)